## 12.6 Avoiding slicing

Recursive algorithms that use the tail operation on arrays
aren't very efficient because it takes linear time to slice an array.

To avoid slicing, we add two parameters to the recursive function:
the start and end indices of the slice to be processed.
Each recursive call reduces the range of the indices instead of
creating a new smaller sequence. In other words, we keep track of the slice
but don't extract it from the sequence. Let's see an example.

### 12.6.1 Problem definition

Consider the [membership function](../04_Iteration/04_1_sequences.ipynb#4.1.1.6-Membership).

**Function**: membership\
**Inputs**: *items*, a sequence; *item*, an object\
**Preconditions**: true\
**Output**: *exists*, a Boolean\
**Postconditions**: *exists* is true if and only if there's an integer *index* such that *items*[*index*] = *item*

A more general membership operation allows the user to specify in which
slice of the input sequence the item should be searched.

**Function**: slice membership\
**Inputs**: *items*, a sequence; *item*, an object; *start*, an integer; *end*, an integer\
**Preconditions**: 0 ≤ *start* ≤ *end* ≤ │*items*│\
**Output**: *exists*, a Boolean\
**Postconditions**: *exists* is true if and only if there's an integer *index* such that *items*[*index*] = *item* and *start* ≤ *index* < *end*

This new operation can be seen as a combination of the
[slicing](../04_Iteration/04_1_sequences.ipynb#4.1.2.1-Slicing) and membership operations.
As such, it has the same preconditions as the slicing operation and
follows the same convention of not including the end item in the slice.

The less general membership function can now be defined more simply:
the item exists in the sequence if it exists in
the slice comprising the whole sequence.

**Function**: membership\
**Inputs**: *items*, a sequence; *item*, an object\
**Preconditions**: true\
**Output**: *exists*, a Boolean\
**Postconditions**: *exists* = slice membership(*items*, *item*, 0, │*items*│)

### 12.6.2 Recursive definition

For the recursive definitions,
I use the shorter function name 'has' instead of 'membership'.

Function has(*items*, *item*) was defined as follows in
[Section&nbsp;12.4.2](../12_Recursion/12_4_inspect_sequences.ipynb#12.4.2-Membership):

- if *items* is empty: has(*items*, *item*) = false
- otherwise if *item* is head(*items*): has(*items*, *item*) = true
- otherwise: has(*items*, *item*) = has(tail(*items*), *item*).

The new function slice\_has(*items*, *item*, *start*, *end*) is defined in
a similar way, but works on the slice given by the start and end indices
instead of the whole sequence.
The first base case is for the slice to be empty.

- if *start* = *end*: slice\_has(*items*, *item*, *start*, *end*) = false

The second base case is for the head of the slice to be the sought item.

- otherwise if *items*[*start*] = *item*: slice\_has(*items*, *item*, *start*, *end*) = true

The recurrence relation looks for the item in the tail of the slice, i.e.
after the current *start* index.

- otherwise: slice\_has(*items*, *item*, *start*, *end*) = slice\_has(*items*, *item*, *start* + 1, *end*)

The recurrence relation is progressing towards the first base case,
as it should:
increasing the start index gets it closer to being equal to the end index.

#### Exercise 12.6.1

Here again is the algorithmic pattern for a
[recursive linear search](../12_Recursion/12_5_create_sequences.ipynb#12.5.2-Linear-search) that outputs
all items from the input sequence *candidates* which satisfy some conditions.

1. if *candidates* is empty:
   1. let *solutions* be ()
2. otherwise:
   1. let *solutions* be search(tail(*candidates*))
   2. if head(*candidates*) satisfies the conditions:
      1. let *solutions* be prepend(head(*candidates*), *solutions*)

Modify the pattern so that it implements the function
search(*candidates*, *start*, *end*), which only searches the slice
*candidates*[*start*:*end*].
The precondition is 0 ≤ *start* ≤ *end* ≤ │*numbers*│.

_Write your answer here._

[Hint](../31_Hints/Hints_12_6_01.ipynb)
[Answer](../32_Answers/Answers_12_6_01.ipynb)

#### Exercise 12.6.2

Here again is the recursive definition of the maximum operation
on a non-empty sequence of numbers.

- if length(*numbers*) = 1: maximum(*numbers*) = head(*numbers*)
- otherwise: maximum(*numbers*) = max(head(*numbers*), maximum(tail(*numbers*)))

Write a recursive definition for a function maximum(*numbers*, *start*, *end*)
that returns the largest number in the slice *numbers*[*start*:*end*].
The precondition is 0 ≤ *start* < *end* ≤ │*numbers*│.

_Write your answer here._

[Hint](../31_Hints/Hints_12_6_02.ipynb)
[Answer](../32_Answers/Answers_12_6_02.ipynb)

### 12.6.3 Code

The algorithm and code directly follow the recursive definition,
so I skip the algorithm.

In [1]:
from algoesup import test


def slice_has(items: list, item: object, start: int, end: int) -> bool:
    """Return True if and only if item is a member of slice items[start:end].

    Preconditions: 0 <= start <= end <= len(items)
    """
    if start == end:
        return False
    elif items[start] == item:
        return True
    else:
        return slice_has(items, item, start + 1, end)


def has(items: list, item: object) -> bool:
    """Return True if and only if item is a member of items."""
    return slice_has(items, item, 0, len(items))


has_tests = [
    # case,         items,   item,  has?
    ('empty list',  [],         3,  False),
    ('last member', [1, 2, 3],  3,  True),
    ('not member',  [1, 2, 3],  4,  False),
]

test(has, has_tests)

Testing has...
Tests finished: 3 passed (100%), 0 failed.


If for some reason we don't want to provide the `slice_has` function
to the user, we can hide it by nesting it inside the main `has` function.
I introduced inner functions in
[Section&nbsp;11.3.3](../11_Search/11_3_constraints.ipynb#11.3.3-Code-and-performance)
to test a candidate for complicated conditions.
An inner function can access the arguments of the outer function.
Here, the sequence being searched and the sought item are never modified,
so we don't have to constantly pass them to each recursive call.
The code becomes slightly shorter.

In [2]:
def has(items: list, item: object) -> bool:
    """Return True if and only if item is a member of items."""

    def in_slice(start: int, end: int) -> bool:
        """Return True if and only if item is in slice items[start:end].

        Preconditions: 0 <= start <= end <= len(items)
        """
        if start == end:
            return False
        elif items[start] == item:
            return True
        else:
            return in_slice(start + 1, end)

    return in_slice(0, len(items))


test(has, has_tests)  # run against the same tests

Testing has...
Tests finished: 3 passed (100%), 0 failed.


#### Exercise 12.6.3

Implement the maximum function, as defined in the previous exercise,
using an inner function.

In [3]:
from algoesup import test


def maximum(numbers: list) -> int:
    """Return the largest number in numbers.

    Preconditions: numbers is a non-empty list of integers
    """
    pass


maximum_tests = [
    # case,             numbers,       maximum
    ('shortest input',  [5],                5),
    ('all equal',       [-1, -1],          -1),
    ('ascending',       [-1, 0, 3],         3),
    ('descending',      [4, 2, 1],          4),
    ('unsorted',        [2, 4, 3, 4, 1],    4)
]

test(maximum, maximum_tests)

[Answer](../32_Answers/Answers_12_6_03.ipynb)

⟵ [Previous section](12_5_create_sequences.ipynb) | [Up](12-introduction.ipynb) | [Next section](12_7_multiple.ipynb) ⟶